In [ ]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from matplotlib.ticker import MaxNLocator
import sys
import torch

# Add module to path
module_path = Path.cwd().parents[1] / "module"  
sys.path.append(str(module_path))

from pool_utils import PINNDataset
from visualization import compute_relative_errors_from_dict

colors = ["tab:red", "tab:blue"]
labels = ["Active Learning", "Random"]
markers = ["s", "o"]

# Path to repo root
repo_root = Path.cwd().parents[1] 

# mode
mode = "convergence"

folder_al = f"7_sensors_{mode}_mode_Noise_0%"
folder_random = f"7_sensors_{mode}_mode_Noise_0%_random"

folder_list = [folder_al, folder_random]

# --- Plot ---
plt.figure(figsize=(8/2.54,6/2.54))

plt.rcParams.update({
    "font.size": 7,
    "axes.labelsize": 7,
    "xtick.labelsize": 7,
    "ytick.labelsize": 7,
    "legend.fontsize": 7
})

for i, folder in enumerate(folder_list):
    
    # Path to data file
    load_dir = os.path.join(repo_root, "examples", f"{mode}_mode", "Results", f"{folder}") 
    save_dir = os.path.join(repo_root, "examples", f"{mode}_mode", "Results", f"{folder}", "posterior_analysis") 
    os.makedirs(save_dir, exist_ok=True)
    
    # List all seed folders
    seed_dirs = [d for d in os.listdir(load_dir) if d.startswith("s_")]
    all_rel_errors = []
    for seed_dir in seed_dirs:
        save_path = os.path.join(load_dir, seed_dir, "all_results.pkl")
        if not os.path.exists(save_path):
            print(f"Warning: {save_path} does not exist, skipping")
            continue
        with open(save_path, "rb") as f:
            results_dict = pickle.load(f)
            
        rel_error_seed = compute_relative_errors_from_dict(results_dict, device=torch.device("cpu"))
    
        # all_rel_errors.append(rel_error_seed[1::2])
        all_rel_errors.append(rel_error_seed[1:])
        
    # Transform into numpy array
    all_rel_errors = np.array(all_rel_errors)
    
    if all_rel_errors.size > 0:
        mean_all_rel_errors = np.nanmean(all_rel_errors, axis=0)
        std_all_rel_errors = np.nanstd(all_rel_errors, axis=0)
    
    else:
        raise ValueError("Error: all_rel_errors array is empty")

    # --- Log-transform des valeurs ---
    log_mean = np.log10(mean_all_rel_errors)
    log_std  = np.log10(mean_all_rel_errors + std_all_rel_errors) - log_mean  
    
    # --- Courbe et bande d'incertitude ---
    x = np.arange(2, mean_all_rel_errors.shape[0] + 2)
    plt.plot(x, 100 * 10**log_mean, markersize=3, marker=markers[i], color=colors[i], label=labels[i], linewidth=1)
    plt.fill_between(
        x,
        100 * 10**(log_mean - log_std),
        100 * 10**(log_mean + log_std),
        color=colors[i], alpha=0.2
    )
    
# --- Mise en forme ---
plt.yscale('log')
plt.xlabel('Number of sensors')
plt.ylabel('Test relative error (%)')
plt.legend()
plt.grid(True, which='major', linestyle='--', linewidth=0.5)
plt.minorticks_on()
plt.grid(True, which='minor', linestyle=':', linewidth=0.3, alpha=0.5)

# Integer x axis
plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
    
# --- Sauvegarde ---
save_path = os.path.join(save_dir, f"AL_vs_Random_{mode}_mode.pdf")
plt.savefig(save_path, bbox_inches='tight', dpi=300)
plt.show()

